# Generación de tablas

In [91]:
import pandas as pd
import requests
import json
import os
from unidecode import unidecode
import re
import pdb

In [92]:
def _df_cols_to_th(df):
    th_cols = ["        <th>{}</th>".format(col) for col in df.columns]
    return "\n".join(th_cols)    

In [93]:
def _df_rows_to_tr(df):

    td_rows = []
    for row in df.iterrows():
        td_row = "\n        ".join([
            "<td>{}</td>".format(element) 
            for element in list(row[1])
        ])
        td_rows.append(td_row)
    
    tr_rows = [
        """
    <tr>
        {}
    </tr>
        """.format(td_row) for td_row in td_rows
    ]

    return "".join(tr_rows)    

In [94]:
def df_to_html(df):
    
    df = df.fillna("")
    
    html="""
<table>
    <tr>
{columns}
    </tr>
{rows}
</table>""".format(
        columns=_df_cols_to_th(df),
        rows=_df_rows_to_tr(df)
    )
    
    return html

In [95]:
def generate_specs_tables(df, resource_field="recurso"):
    text_elements = []

    for recurso in df[resource_field].unique():
        text_elements.append("### Recurso: {}".format(recurso.ljust(10)))
        text_elements.append(df_to_html(
            df[df.recurso == recurso].drop("recurso", axis=1)
        ))

    return "\n".join(text_elements)

In [96]:
def get_example_from_fields(df, filename):
    df_transposed = df[["titulo", "ejemplo"]].transpose()
    df_transposed.columns = df.titulo
    df_transposed = df_transposed.drop("titulo")
    
    df_transposed.to_csv(filename, encoding="utf8", index=False)
    df_transposed.to_excel(filename.replace(".csv", ".xlsx")
                           , encoding="utf8", index=False)
    
    return df_transposed

In [97]:
def title_to_name(title, decode=True, max_len=None, use_complete_words=True):
    """Convierte un título en un nombre normalizado para generar urls."""
    # decodifica y pasa a minúsculas
    if decode:
        title = unidecode(title)
    title = title.lower()

    # remueve caracteres no permitidos
    filtered_title = re.sub(r'[^a-z0-9- ]+', '', title)

    # remueve stop words y espacios y une palabras sólo con un "-"
    normalized_title = '-'.join([word for word in filtered_title.split()
                                 if word not in STOP_WORDS])

    # recorto el titulo normalizado si excede la longitud máxima
    if max_len and len(normalized_title) > max_len:

        # busco la última palabra completa
        if use_complete_words:
            last_word_index = normalized_title.rindex("-", 0, max_len)
            normalized_title = normalized_title[:last_word_index]

        # corto en el último caracter
        else:
            normalized_title = normalized_title[:max_len]

    return normalized_title

In [98]:
STOP_WORDS = [
    "el", "la", "los", "las",
    "de", "del",
    "y", "a",
    "un", "una", "en"
]

In [99]:
def generate_example(df, specs_name, resource_field="recurso"):
    
    if not os.path.exists(specs_name):
        os.makedirs(specs_name)
    
    text_elements = []
    for recurso in df[resource_field].unique():
        example_filename = os.path.join(
            specs_name,
            title_to_name(recurso) + ".csv"
        )
        
        text_elements.append("### Recurso: {}".format(recurso.ljust(10)))
        text_elements.append("**[CSV]({})** | **[XLSX]({})**".format(
            example_filename,
            example_filename.replace(".csv", ".xlsx")
        ))
        text_elements.append(df_to_html(
            get_example_from_fields(
                df[df.recurso == recurso].drop("recurso", axis=1),
                example_filename
            )
        ))

    return "\n".join(text_elements)

In [100]:
def generate_example_section(specs_name):
    specs_fields_csv = specs_name + "-campos.csv"
    
    df = pd.read_csv(specs_fields_csv)
    md_example_tables = generate_example(df, specs_name)
    
    md_example = """## Ejemplos
    
{}""".format(md_example_tables)
    
    return md_example

In [101]:
def generate_field_section(specs_name):
    specs_fields_csv = specs_name + "-campos.csv"
    specs_fields_xlsx = specs_name + "-campos.xlsx"
    
    df = pd.read_csv(specs_fields_csv)
    df.to_excel(specs_fields_xlsx, encoding="utf8", index=False)
    
    fields_section = """## Campos

Descargar campos en **[CSV]({})** | **[XLSX]({})**

{}""".format(
        specs_fields_csv, 
        specs_fields_xlsx,
        generate_specs_tables(df)
    )
    
    return fields_section

In [102]:
def generate_class_section(specs_name, class_explain):
    specs_example = specs_name + ".csv"
    specs_class_csv = specs_name + "-clases.csv"
    specs_class_xlsx = specs_name + "-clases.xlsx"
    
    df = pd.read_csv(specs_class_csv)
    df.to_excel(specs_class_xlsx, encoding="utf8", index=False)
    
    class_section = """## Clases

{}

Descargar clases en **[CSV]({})** | **[XLSX]({})**

{}""".format(
        class_explain,
        specs_class_csv, 
        specs_class_xlsx,
        df_to_html(df)
    )
    
    return class_section

In [103]:
def download_specs(specs_name, url, suffix):
    specs_filename = "{}-{}.csv".format(specs_name, suffix)

    print("Descargando {}...".format(specs_filename))
    r = requests.get(url, allow_redirects=True)
    with open(specs_filename, 'wb') as f:
        f.write(r.content)

In [104]:
def replace_field_section(specs_md, md_field, all_params):
    before_tag = all_params["tags"]["field_before"]
    after_tag = all_params["tags"]["field_after"]
   
    before_text = specs_md.split(before_tag)[0]
    after_text = specs_md.split(after_tag)[1]
    
    new_specs_md = """{before_text}{before_tag}

{md_field}

{after_tag}{after_text}""".format(
        before_text=before_text,
        before_tag=before_tag,
        md_field=md_field,
        after_tag=after_tag,
        after_text=after_text
    )
    
    return new_specs_md

In [105]:
def replace_class_section(specs_md, md_class, all_params):
    before_tag = all_params["tags"]["class_before"]
    after_tag = all_params["tags"]["class_after"]
   
    before_text = specs_md.split(before_tag)[0]
    after_text = specs_md.split(after_tag)[1]
    
    new_specs_md = """{before_text}{before_tag}

{md_class}

{after_tag}{after_text}""".format(
        before_text=before_text,
        before_tag=before_tag,
        md_class=md_class,
        after_tag=after_tag,
        after_text=after_text
    )
    
    return new_specs_md

In [106]:
def replace_example_table(specs_md, md_example, all_params):
    before_tag = all_params["tags"]["example_before"]
    after_tag = all_params["tags"]["example_after"]
   
    before_text = specs_md.split(before_tag)[0]
    after_text = specs_md.split(after_tag)[1]
    
    new_specs_md = """{before_text}{before_tag}

{md_example}

{after_tag}{after_text}""".format(
        before_text=before_text,
        before_tag=before_tag,
        md_example=md_example,
        after_tag=after_tag,
        after_text=after_text
    )
    
    return new_specs_md

In [107]:
def update_dataset_specs(specs_name, config="config.json"):

    # lee el archivo de configuración
    with open("config.json", "r") as f:
        all_params = json.load(f)
        specs_params = all_params["specs"][specs_name]
    
    # descarga las tablas de campos y clases de Google Spreadsheet en CSV
    download_specs(specs_name, specs_params["gsheet_fields"], "campos")
    if "gsheet_classes" in specs_params:
        download_specs(specs_name, specs_params["gsheet_classes"], "clases")
    
    # re-genera las secciones en base a los CSVs descargados
    md_example = generate_example_section(specs_name)
    if "gsheet_classes" in specs_params:
        md_class = generate_class_section(specs_name, specs_params["class_explain"])
    md_field = generate_field_section(specs_name)
    
    # reemplaza las secciones viejas por las nuevas
    with open(specs_name + ".md", "r") as f:
        specs_md = f.read()
    
    specs_md = replace_example_table(specs_md, md_example, all_params)
    if "gsheet_classes" in specs_params:
        specs_md = replace_class_section(specs_md, md_class, all_params)
    specs_md = replace_field_section(specs_md, md_field, all_params)
    
    # escribe el .md de especificación actualizado
    with open(specs_name + ".md", "w") as f:
        f.write(specs_md)

In [113]:
def main(): 
    # lee el archivo de configuración
    with open("config.json", "r") as f:
        params = json.load(f)
    
    for specs_name in params["specs"]:
        print("=== Actualizando {} ===".format(specs_name))
        try:
            update_dataset_specs(specs_name)
        except Exception as e:
            print(e)
        print()

In [109]:
%pdb

Automatic pdb calling has been turned OFF


In [115]:
update_dataset_specs("contrataciones-publicas")

Descargando contrataciones-publicas-campos.csv...


KeyError: 'recurso'

In [114]:
main()

=== Actualizando nomina-funcionarios ===
Descargando nomina-funcionarios-campos.csv...
Descargando nomina-funcionarios-clases.csv...

=== Actualizando declaraciones-juradas ===
Descargando declaraciones-juradas-campos.csv...

=== Actualizando contrataciones-publicas ===
Descargando contrataciones-publicas-campos.csv...
'recurso'

=== Actualizando presupuesto ===
Descargando presupuesto-campos.csv...

=== Actualizando educacion ===
Descargando educacion-campos.csv...

=== Actualizando denuncias ===
Descargando denuncias-campos.csv...

=== Actualizando organigrama ===
Descargando organigrama-campos.csv...

=== Actualizando permisos ===
Descargando permisos-campos.csv...
'recurso'

=== Actualizando registro-proveedores ===
Descargando registro-proveedores-campos.csv...

=== Actualizando elecciones ===
Descargando elecciones-campos.csv...

=== Actualizando servicios ===
Descargando servicios-campos.csv...

=== Actualizando tramites ===
Descargando tramites-campos.csv...

=== Actualizando c